In [ ]:
# Import the necessary libraries 
import pandas as pd
from matplotlib import pyplot as plt
from reservoirpy.nodes import Reservoir, Ridge, Input
import numpy as np
from reservoirpy.observables import nrmse, rsquare
import json
from reservoirpy.hyper import research
from reservoirpy.hyper import plot_hyperopt_report
from reservoirpy.observables import rmse, rsquare, nrmse

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import scipy.stats as stats

In [ ]:
# Load the data
emg_data_train = pd.read_csv('./Kinematics_Train_Data_EMG.csv')
emg_data_test = pd.read_csv('./Kinematics_Test_Data_EMG.csv')
angles_data_train = pd.read_csv('./Kinematics_Train_Data_Angles.csv')
angles_data_test = pd.read_csv('./Kinematics_Test_Data_Angles.csv')

In [ ]:
# Calculate the number of subjects in the training data
emg_data_train_test = np.array(emg_data_train)
data_per_subject = 101
n_subjects = int (emg_data_train_test.shape[0]/data_per_subject)

# Reshape EMG and angles training data
emg_data_train_test = emg_data_train_test.reshape(n_subjects, data_per_subject, 16)
angles_data_train_test = np.array(angles_data_train)
angles_data_train_test = angles_data_train_test.reshape(n_subjects, data_per_subject, 20)

# Define the number of output angles to predict
n_angles = 17
angles_data_train_test = angles_data_train_test[:, :, 0:n_angles]

# Reshape EMG testing data
emg_data_train_test_ev = np.array(emg_data_train)
emg_data_train_test_ev

# Reshape angles testing data
emg_data_test_test = np.array(emg_data_test)
emg_data_test_test = emg_data_test_test.reshape(n_subjects, data_per_subject, 16)

#emg_data_test_test = emg_data_test_test.reshape(14, data_per_subject, 16)
angles_data_test_test = np.array(angles_data_test)
angles_data_test_test = angles_data_test_test.reshape(n_subjects, data_per_subject, 20)
angles_data_test_test = angles_data_test_test[:, :, 0:n_angles]

# Assign data to variables
x_train = emg_data_train_test
y_train = angles_data_train_test
x_test = emg_data_test_test
y_test = angles_data_test_test
x_train_ev = emg_data_train_test_ev

In [ ]:
# Define hyperparameters
batch_size = 1
input_dim = (data_per_subject, 16)
units = 512
output_size = 101  # labels are from 0 to 9

# Build the RNN model
def build_model(allow_cudnn_kernel=True):
    model = keras.models.Sequential([
        keras.layers.SimpleRNN(units, input_shape=(data_per_subject, 16)),
        keras.layers.Embedding(input_dim=data_per_subject, output_dim=8),
        keras.layers.BatchNormalization(),
        tf.keras.layers.Conv1D(n_angles, units - (data_per_subject - 1)),
        keras.layers.Dense(n_angles),
    ])
    return model

# Create and display the model
model = build_model()
model.summary()


In [ ]:
# Build the model
model = build_model(allow_cudnn_kernel=True)

# Compile the model
model.compile(
    loss=keras.losses.MeanAbsoluteError(),
    optimizer="sgd",
    metrics=["accuracy"],
)

# Train the model
model.fit(x_train, y_train, batch_size=batch_size, epochs=200)


In [ ]:
# Calculate Mean Squared Error (MSE) for training and test data
mse = tf.keras.losses.MeanSquaredError()

mse_train = mse (model.predict(x_train), y_train)
mse_test = mse (model.predict(x_test), y_test)

# Calculate Root Mean Squared Error (RMSE)
rmse_train = mse_train**0.5
rmse_test = mse_test**0.5

In [ ]:
Array1 = y_train
Array2 = model.predict_on_batch(x_train)
resultArray = np.ones(n_subjects*data_per_subject*n_angles).reshape(n_subjects,data_per_subject,n_angles)

for i in range(Array1.shape[0]):
        for j in range(Array1.shape[1]):
            for k in range(Array1.shape[2]):
                slope, intercept, r_value, p_value, std_err = stats.linregress(
                    Array1[i, j, :],
                    Array2[i, j, :]
                )
                R2 = r_value ** 2
                resultArray[ i , j , k] = R2

mean_R2 = np.mean(resultArray)